In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
os.chdir("..")
from model import run_experiments
from model.config.params import build_params
import numpy as np

df, simulation_kpis = run_experiments(["Base", "Base"])


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (2, 730, 128, 2, 25)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (365, 64, 1, 25)
     Simulation 1: (Timesteps, Params, Runs, Sub-States) = (365, 64, 1, 25)
Execution Method: local_simulations
Execution Mode: parallelized
2023-12-12 18:32:51.030375
2023-12-12 18:32:51.036756
Total execution time: 25.24s


In [2]:
servicer_npvs = {}
for key in unique_servicers:
    servicers = unique_servicers[key]
    t = simulation_kpis.loc[key, "timestep"]
    p_j = 1 - (1 - simulation_kpis.loc[key, "param_servicer_jailing_probability"]) ** t
    # For now slashing only comes out of jailing so setting equal
    p_s = p_j
    

NameError: name 'unique_servicers' is not defined

## Basic Line Graphs

In [ ]:
for col in ['total_relays', 'processed_relays', 'pokt_price_true', 'pokt_price_oracle',
           'n_transactions', 'floating_supply', 'n_understaked_servicers', 'n_understaked_gateways',
            'n_understaked_applications', 'POKT_burned', 'POKT_minted','period_slashing_costs',
            'period_jailing_opportunity_cost','POKT_net_mint', 'total_application_stake',
            'total_servicer_stake', 'total_gateway_stake', 'total_stake', 'circulating_supply',
            'dao_value_capture', 'n_servicers', 'n_applications', 'n_gateways',
           'n_services', 'kpi_a', 'kpi_c']:
    # Grab the pivot table
    table = df.pivot("timestep", "key", col)
    
    # Plot
    table.plot(kind = 'line')
    plt.xlabel("timestep")
    plt.ylabel(col)
    plt.title("Simulation Values for {}".format(col))
    
    plt.show()